<a href="https://colab.research.google.com/github/Phurin216/Datawarehouse/blob/main/%E0%B8%AA%E0%B8%A3%E0%B8%B8%E0%B8%9BCoding_Data_warehouse_%E0%B8%99%E0%B8%B2%E0%B8%A2%E0%B8%A0%E0%B8%B9%E0%B8%A3%E0%B8%B4%E0%B8%99%E0%B8%97%E0%B9%8C_%E0%B8%AA%E0%B8%B2%E0%B8%A2%E0%B9%80%E0%B8%8A%E0%B8%B7%E0%B9%89%E0%B8%AD_653020216_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#สรุปโค้ด Datawarehouse Midterm
นายภูรินท์ สายเชื้อ 653020216-6

In [ ]:
#Def function

def add_timestamp(df,colname):
now = datetime.now()
current_timestamp = now.strftime('%m/%d/%Y %H:%M:%S')
df_new = (df.with_columns((pl.lit(current_timestamp)).alias(colname)
)
.pipe(change_dtypedatetime,colname)
)
return df_new
def change_dtypedatetime(df,colname):
format = '%m/%d/%Y %H:%M:%S'
return df.with_columns(
pl.col(colname).str.strptime(pl.Date,format=format)
)

In [ ]:
#โมดูลในการวิเคราะห์

import pandas as pd
import polars as pl
import numpy as np
import duckdb as dd
import dlt
import seaborn as sns
import streamlit as st
import sqlite3 as sq
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
#Dashboard

import streamlit as st
import pandas as pd
import plotly.express as px

# Sample sales data with two measures (sales and quantity)
data = pd.DataFrame({
'region': ['North America', 'North America', 'North America', 'South America', 'South America', 'Europe', 'Europe', 'Asia', 'Asia'],
'country': ['USA', 'Canada', 'Mexico', 'Brazil', 'Argentina', 'Germany', 'France', 'China', 'India'],
'product': ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'E'],
'sales': [100, 150, 200, 250, 300, 350, 400, 450, 500],
'quantity': [10, 15, 20, 25, 30, 35, 40, 45, 50]
})

In [ ]:
#Import duckdb And database

name_db = 'load_nw2dw.duckdb'
conn = dd.connect(name_db)
dim_customers = pl.read_database(
query= "SELECT * FROM northwind_dw.dim_customers",
connection=conn
)
dim_time = pl.read_database(
query= "SELECT * FROM northwind_dw.dim_date",
connection=conn
)
fact_sales = pl.read_database(
query= "SELECT * FROM northwind_dw.fact_sale",
connection=conn
)

dd.commit
dd.close()

In [ ]:
#สำหรับ staging database

pd_suppliers = (pl.read_csv('./data/northwind/suppliers.csv')
.pipe(recolname,{'id':'supplier_id'})
.pipe(add_timestamp,'ingestion_timestamp')
.to_pandas()
)

pd_orders = (pl.read_csv('./data/northwind/orders.csv')
.pipe(add_timestamp,'ingestion_timestamp')
.pipe(recolname,{'id':'order_id'})
.pipe(sort_by,'order_id')
.to_pandas()
)

pd_orders_details = (pl.read_csv('./data/northwind/order_details.csv')
.pipe(add_timestamp,'ingestion_timestamp')
.pipe(recolname,{'id':'orderdetail_id'})
.to_pandas()
)

In [ ]:
#Read Stg

- Read `stg_orders` tabel with `northwind_stg` schema from `load_northwind.duckdb` and naming `df_order`.
- Read `stg_orders_details` tabel with `northwind_stg` schema from `load_northwind.duckdb` and naming `df_order_detail`.
- From `df_order` table, select columns: ['order_id','customer_id','employee_id','shipper_id',
'order_date','shipped_date','paid_date']
- Change dtype of `order_date` string type to `Date` dtype.
- From `df_order_detail` table, select columns: ['order_id','product_id',
'status_id','purchase_order_id','inventory_id','quantity','unit_price','discount']
- Perform left join `df_order_detail` table to `df_order` and naming `fact_table`.
- Add timstamp column with column name `insertion_timestamp`.

In [ ]:
#Setting data base

## 1.Setting start and end dates
start_date = pl.datetime(2006, 1, 1)
end_date = pl.datetime(2050, 1, 1)

## 2.Generate date range
date_range = pl.date_range(start=start_date, end=end_date, interval="1d", eager=True)

## 3.Convert to DataFrame and setting column name as 'date'
df_date = pl.DataFrame({
"date": date_range
})

In [ ]:
#่Date time


pd_date = (df_date.select(
pl.col("date").dt.strftime("%m-%d-%Y").alias("date_id"),
pl.col("date").dt.date().alias("order_date"),
pl.col("date").dt.year().alias("year"),
pl.col("date").dt.week().alias("year_week"),
pl.col("date").dt.ordinal_day().alias("year_day"),
pl.col("date").dt.year().alias("fiscal_year"),
pl.col("date").dt.quarter().alias("fiscal_qtr"),
pl.col("date").dt.strftime("%B").alias("month_name"),
pl.col("date").dt.strftime("%W").alias("week_day"),
pl.col("date").dt.strftime("%A").alias("day_name"),
).with_columns(pl.col('day_name').is_in(['Saturday','Sunday']).alias("WeekendFlag"))
.to_pandas()
)